In [8]:
LLM1 = "llama3"
LLM2 = "phi3"
CHAT_LENGTH = 3
STARTING_PROMPT = "I think climate change is a hoax because it's cold during the winter."
SYSTEM_PROMPT = """You are a participant in a debate. You have to argue about a given topic by reacting to your opponent's text. 
You have to give concise answers to your opponent and concede when you feel appropriate, 
for example if you can't defend your initial argument from a counter, or otherwise lost the argument. 
You must not argue against your own side with novel information that you give, you can only do that by accepting your opponent's counterargument against your own. 
Be sure to ignore politeness and formalities, no need to comment on how good a question or how insightful an argument is if it doesn't contribute to the reasoning process. 
Do not ask back what the other person would like to talk about, if you have reason to say something (like countering their argument or protecting your own) then you're free to do so."""
CLOSING = True
CLOSING_PROMPT = "This is your last turn, give your closing arguments on the topic."

url = 'http://127.0.0.1:11434/api/chat'

In [9]:
import requests

def post(json):
    r = requests.post(url=url, json=json)
    if r.status_code == 200:
        return True, r.json()["message"]["content"]
    else:
        return False, r.text

def build_json(model, messages, first = True):
    return {
        "model": model,
        "messages": [{"role": "system", "content": SYSTEM_PROMPT}] + [{"role": "user" if (i + int(first)) % 2 == 1 else "assistant", "content": message} for i, message in enumerate(messages)],
        "stream": False
    }

In [10]:
messages = [STARTING_PROMPT]
print("Starting Prompt: ")
print(messages[-1])
print()
for i in range(CHAT_LENGTH):
    succ, resp = post(build_json(LLM1, messages))
    if succ:
        messages.append(resp)
        print("\nLLM1:")
        print(messages[-1])
    else:
        break
    succ, resp = post(build_json(LLM2, messages, False))
    if succ:
        messages.append(resp)
        print("\nLLM2:")
        print(messages[-1])
    else:
        break

if CLOSING:    
    closing_llm1 = build_json(LLM1, messages)
    closing_llm1["messages"].append({"role": "user", "content": CLOSING_PROMPT})
    succ, resp = post(closing_llm1)
    if succ:
        print("\nLLM1:")
        print(resp)

    messages.append(CLOSING_PROMPT)
    succ, resp = post(build_json(LLM2, messages, False))
    if succ:
        print("\nLLM2:")
        print(resp)

Starting Prompt: 
I think climate change is a hoax because it's cold during the winter.


LLM1:
That's an oversimplification of a complex issue. Just because it's cold in the winter doesn't mean that the overall trend of global warming isn't happening. Climate change is not just about temperature, but also about patterns and extremes. Even if it's cold in your area during the winter, other parts of the world are experiencing record-breaking heatwaves, droughts, or heavy rainfall.

Moreover, scientists have been studying climate change for decades and their research has consistently shown that human activities like burning fossil fuels, deforestation, and pollution are causing the Earth's temperature to rise. The evidence is overwhelming: melting glaciers, sea-level rise, shifts in seasons, and changes in weather patterns.

It's also important to note that climate change is not a hoax or a conspiracy theory. It's a scientific fact based on empirical data and observations. Climate scient